In [12]:
from bs4 import BeautifulSoup
import requests

In [13]:
# Step 2: Fetch the content of the page
url = "https://dlr.concludis.de/prj/lst/a181a603769c1f98ad927e7367c7aa51/GesamtlisteOffenePositionen.htm?b=0&boerse=&stellg1=&stellg2=&stellort=&page=1"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Page fetched successfully")
    html_content = response.content
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Page fetched successfully


In [16]:
soup=BeautifulSoup(html_content,"html.parser")
print(soup.prettify()[:15000])

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="de">
 <head>
  <title>
   Ihre Karriere beim DLR - Offene Stellen
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html; utf-8" http-equiv="Content-Type">
   <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
   <meta content="width=device-width, initial-scale=1" name="viewport"/>
   <meta content="Ihre Karriere beim DLR" name="description">
    <meta content="Deutsches Zentrum für Luft- und Raumfahrt e. V. (DLR), Stellenbörse, Jobbörse, Stellen, Stellenangebote" name="keywords">
     <meta content="index, follow, noarchive" name="robots"/>
     <meta content="Stellenbörse" name="page-topic"/>
     <meta content="All" name="audience"/>
     <meta content="Global" name="distribution"/>
     <meta content="1 day" name="revisit-after"/>
     <meta content="de" http-equiv="Content-Language"/>
     <link href="https://dlr.concludis.de/favicon.ico" rel="shortcut ic

In [26]:
# Step 5: Extract job titles and application links from the listings
# Find all job listings that have the class "line_0" or similar (based on your provided structure)

job_listings = soup.find_all("div", class_="line_0")  # Adjust the class if necessary

# Initialize a list to store the job titles and links
jobs = []

# Loop through each job listing and extract the required information
for job in job_listings:
    # Extract job title from the 'span' with class 'headerlink stellenlink'
    job_title = job.find("span", class_="headerlink stellenlink").get_text(strip=True)
    
    # Extract application link from the 'onclick' attribute
    onclick_text = job['onclick']
    job_link = onclick_text.split("'")[1]  # Extract the URL from the onclick attribute
    
    # Extract the job description (optional if needed)
    job_description = job.find("span", class_="kurzb").get_text(strip=True)
    
    # Append the extracted job data to the jobs list
    jobs.append({
        'title': job_title,
        'link': job_link,
        'description': job_description
    })

# Print the extracted jobs
for job in jobs:
    print(f"Title: {job['title']}\nLink: {job['link']}\nDescription: {job['description']}\n")


Title: Fachinformatiker/in für Systemintegration (w/m/d)
Link: https://dlr.concludis.de/prj/shw/823f1eb01fb652a660e7c73b33932495_0/97774/Fachinformatiker_in_fuer_Systemintegration_w_m_d.htm?b=0
Description: Für das Technische Facility Management in Oberpfaffenhofen suchen wir eine/n Fachinformatiker/in für Systemintegration (w/m/d),

Title: Konstruktionsingenieur/in (w/m/d) - Maschinenbau, Luft- und Raumfahrttechnik o. ä.
Link: https://dlr.concludis.de/prj/shw/0c90629526d274353aa0d0f2f3c4eea7_0/97594/Konstruktionsingenieur_in_w_m_d_-_Maschinenbau_Luft-_und_Raumfahrttechnik_o_ae_.htm?b=0
Description: Für unser Institut für Antriebstechnik in Köln suchen wir eine/n Konstruktionsingenieur/in (w/m/d) - Maschinenbau, Luft- und Raumfahrttechnik o. ä., Realisierung experimenteller Versuchsträger auf dem Gebiet der Brennkammerforschung in komplexen Forschungsanlagen

Title: Verwaltungs(fach)wirt/in oder Betriebswirt/in oder Kauffrau/mann (Diplom FH/Bachelor) (w/m/d)
Link: https://dlr.concludis

In [28]:
# Import additional libraries if not already imported
from bs4 import BeautifulSoup
import requests

# Function to fetch job description from the respective job link
def fetch_job_description(job_url):
    try:
        # Make a request to the job's detailed page
        response = requests.get(job_url)
        if response.status_code == 200:
            job_soup = BeautifulSoup(response.content, 'html.parser')
            
            # Initialize an empty description
            description = ""

            # Extract content from the relevant sections (con_11, con_4, con_12, con_5, con_15)
            for section in ['con_11', 'con_4', 'con_12', 'con_5', 'con_15']:
                content = job_soup.find("div", id=section)
                if content:
                    description += content.get_text(strip=True) + "\n"
            
            return description.strip() if description else "No detailed description available"
        else:
            return "Failed to load job details"
    except Exception as e:
        return f"Error fetching description: {e}"

# Step 6: Extract job titles, application links, and fetch detailed job descriptions
job_listings = soup.find_all("div", class_="line_0")  # Adjust the class if necessary

# Initialize a list to store the job titles, links, and descriptions
jobs = []

# Loop through each job listing and extract the required information
for job in job_listings:
    # Extract job title from the 'span' with class 'headerlink stellenlink'
    job_title = job.find("span", class_="headerlink stellenlink").get_text(strip=True)
    
    # Extract application link from the 'onclick' attribute
    onclick_text = job['onclick']
    job_link = onclick_text.split("'")[1]  # Extract the URL from the onclick attribute
    
    # Fetch the detailed job description by visiting the job link
    job_description = fetch_job_description(job_link)
    
    # Append the extracted job data to the jobs list
    jobs.append({
        'title': job_title,
        'link': job_link,
        'description': job_description
    })

# Print the extracted jobs (including descriptions)
for job in jobs:
    print(f"Title: {job['title']}\nLink: {job['link']}\nDescription: {job['description']}\n")


Title: Fachinformatiker/in für Systemintegration (w/m/d)
Link: https://dlr.concludis.de/prj/shw/823f1eb01fb652a660e7c73b33932495_0/97774/Fachinformatiker_in_fuer_Systemintegration_w_m_d.htm?b=0
Description: Das erwartet Sie:
Die Technische Infrastruktur (TI) agiert als interner Dienstleister im DLR. Das Facility Management (FM) als Teil der TI baut und betreibt Gebäude und Einrichtungen des DLR und liefert so einen wertvollen Beitrag zum effizienten und produktiven Forschungsbetrieb und somit zum wissenschaftlichen Erfolg des DLR.Sie wollen mehr über das DLR wissen? Dann klicken Sie gernehier.Das Technische Facility Management (TFM) als Teil des FM an den Standorten in der Region Süd (Oberpfaffenhofen, Augsburg, Weilheim) unterstützt die wissenschaftlichen, häufig einmaligen, hochkomplexen Großversuchsanlagen, wie z. B. das Raumfahrtkontrollzentrum.Zu den Aufgaben des TFM zählen u. a.:Bauen und Betreiben von Gebäuden, Anlagen und InfrastrukturGebäude- und AnlagendokumentationPlanung, G

In [31]:
# Function to fetch job description, type, and misc details from the respective job link
def fetch_job_details(job_url):
    try:
        # Make a request to the job's detailed page
        response = requests.get(job_url)
        if response.status_code == 200:
            job_soup = BeautifulSoup(response.content, 'html.parser')
            
            # Initialize empty fields for description, type, and misc
            description = ""
            job_type = ""
            misc = ""

            # Extract content from the relevant sections (con_11, con_4, con_12, con_5, con_15 for description)
            for section in ['con_11', 'con_4', 'con_12', 'con_5', 'con_15']:
                content = job_soup.find("div", id=section)
                if content:
                    description += content.get_text(strip=True) + "\n"
            
            # Extract job type from con_8
            job_type_content = job_soup.find("div", id="con_8")
            if job_type_content:
                job_type = job_type_content.get_text(strip=True)
            else:
                job_type = "No job type available"
            
            # Extract misc from con_7
            misc_content = job_soup.find("div", id="con_7")
            if misc_content:
                misc = misc_content.get_text(strip=True)
            else:
                misc = "No additional information available"
            
            return description.strip() if description else "No detailed description available", job_type, misc
        else:
            return "Failed to load job details", "Failed to load job type", "Failed to load misc"
    except Exception as e:
        return f"Error fetching description: {e}", "Error fetching job type", "Error fetching misc"

# Step 8: Extract job titles, application links, job descriptions, job types, and misc details
job_listings = soup.find_all("div", class_="line_0")  # Adjust the class if necessary

# Initialize a list to store the job titles, links, descriptions, types, and misc
jobs = []

# Loop through each job listing and extract the required information
for job in job_listings:
    # Extract job title from the 'span' with class 'headerlink stellenlink'
    job_title = job.find("span", class_="headerlink stellenlink").get_text(strip=True)
    
    # Extract application link from the 'onclick' attribute
    onclick_text = job['onclick']
    job_link = onclick_text.split("'")[1]  # Extract the URL from the onclick attribute
    
    # Fetch the detailed job description, job type, and misc info by visiting the job link
    job_description, job_type, job_misc = fetch_job_details(job_link)
    
    # Append the extracted job data to the jobs list
    jobs.append({
        'title': job_title,
        'link': job_link,
        'description': job_description,
        'type': job_type,   # Adding job type
        'misc': job_misc    # Adding misc info
    })

# Print the extracted jobs (including descriptions, job types, and misc details)
for job in jobs:
    print(f"Title: {job['title']}\nLink: {job['link']}\nDescription: {job['description']}\nType: {job['type']}\nMisc: {job['misc']}\n")


Title: Fachinformatiker/in für Systemintegration (w/m/d)
Link: https://dlr.concludis.de/prj/shw/823f1eb01fb652a660e7c73b33932495_0/97774/Fachinformatiker_in_fuer_Systemintegration_w_m_d.htm?b=0
Description: Das erwartet Sie:
Die Technische Infrastruktur (TI) agiert als interner Dienstleister im DLR. Das Facility Management (FM) als Teil der TI baut und betreibt Gebäude und Einrichtungen des DLR und liefert so einen wertvollen Beitrag zum effizienten und produktiven Forschungsbetrieb und somit zum wissenschaftlichen Erfolg des DLR.Sie wollen mehr über das DLR wissen? Dann klicken Sie gernehier.Das Technische Facility Management (TFM) als Teil des FM an den Standorten in der Region Süd (Oberpfaffenhofen, Augsburg, Weilheim) unterstützt die wissenschaftlichen, häufig einmaligen, hochkomplexen Großversuchsanlagen, wie z. B. das Raumfahrtkontrollzentrum.Zu den Aufgaben des TFM zählen u. a.:Bauen und Betreiben von Gebäuden, Anlagen und InfrastrukturGebäude- und AnlagendokumentationPlanung, G

In [32]:
import math
from bs4 import BeautifulSoup
import requests
import csv
import time
import random

# Function to fetch job description, type, and misc details from the respective job link
def fetch_job_details(job_url, headers):
    try:
        # Make a request to the job's detailed page
        response = requests.get(job_url, headers=headers)
        if response.status_code == 200:
            job_soup = BeautifulSoup(response.content, 'html.parser')
            
            # Initialize empty fields for description, type, and misc
            description = ""
            job_type = ""
            misc = ""

            # Extract content from the relevant sections (con_11, con_4, con_12, con_5, con_15 for description)
            for section in ['con_11', 'con_4', 'con_12', 'con_5', 'con_15']:
                content = job_soup.find("div", id=section)
                if content:
                    description += content.get_text(strip=True) + "\n"
            
            # Extract job type from con_8
            job_type_content = job_soup.find("div", id="con_8")
            if job_type_content:
                job_type = job_type_content.get_text(strip=True)
            else:
                job_type = "No job type available"
            
            # Extract misc from con_7
            misc_content = job_soup.find("div", id="con_7")
            if misc_content:
                misc = misc_content.get_text(strip=True)
            else:
                misc = "No additional information available"
            
            return description.strip() if description else "No detailed description available", job_type, misc
        else:
            return "Failed to load job details", "Failed to load job type", "Failed to load misc"
    except Exception as e:
        return f"Error fetching description: {e}", "Error fetching job type", "Error fetching misc"

# Function to fetch all jobs from a single page
def fetch_jobs_from_page(page_url, headers):
    response = requests.get(page_url, headers=headers)
    if response.status_code == 200:
        page_soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all job listings on the page (adjust the class based on the actual structure)
        job_listings = page_soup.find_all("div", class_="line_0")
        
        # Initialize a list to store jobs from the current page
        jobs = []
        
        # Loop through each job listing and extract job details
        for job in job_listings:
            # Extract job title
            job_title = job.find("span", class_="headerlink stellenlink").get_text(strip=True)
            
            # Extract application link
            onclick_text = job['onclick']
            job_link = onclick_text.split("'")[1]  # Extract the URL from the onclick attribute
            
            # Fetch job details (description, type, misc)
            job_description, job_type, job_misc = fetch_job_details(job_link, headers)
            
            # Append the job data to the list
            jobs.append({
                'title': job_title,
                'link': job_link,
                'description': job_description,
                'type': job_type,
                'misc': job_misc
            })
        
        return jobs
    else:
        print(f"Failed to load page: {page_url}")
        return []

# Base URL for the first page
base_url = "https://dlr.concludis.de/prj/lst/a181a603769c1f98ad927e7367c7aa51/GesamtlisteOffenePositionen.htm?b=0&boerse=&stellg1=&stellg2=&stellort=&page="

# Total number of pages
total_jobs = 599
jobs_per_page = 25
total_pages = math.ceil(total_jobs / jobs_per_page)  # 24 pages

# Set custom headers to simulate browser behavior
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Initialize an empty list to store all jobs
all_jobs = []

# Loop through all the pages and fetch jobs from each page
for page in range(1, total_pages + 1):  # Pages 1 to 24
    page_url = f"{base_url}{page}"
    print(f"Scraping page {page} of {total_pages}: {page_url}")
    
    # Fetch jobs from the current page
    jobs_on_page = fetch_jobs_from_page(page_url, headers)
    
    # Add the jobs from the current page to the all_jobs list
    all_jobs.extend(jobs_on_page)
    
    # Introduce a random delay between 5 and 10 seconds before the next request
    delay = random.uniform(2, 7)
    print(f"Waiting for {delay:.2f} seconds before scraping the next page...")
    time.sleep(delay)

# Define the CSV file path
csv_file_path = 'job_listings.csv'

# Write the data to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['title', 'link', 'description', 'type', 'misc'])
    
    # Write the header
    writer.writeheader()
    
    # Write all job rows to the CSV file
    for job in all_jobs:
        writer.writerow(job)

print(f"Data scraping complete! All job listings have been written to {csv_file_path}")


Scraping page 1 of 24: https://dlr.concludis.de/prj/lst/a181a603769c1f98ad927e7367c7aa51/GesamtlisteOffenePositionen.htm?b=0&boerse=&stellg1=&stellg2=&stellort=&page=1
Waiting for 5.05 seconds before scraping the next page...
Scraping page 2 of 24: https://dlr.concludis.de/prj/lst/a181a603769c1f98ad927e7367c7aa51/GesamtlisteOffenePositionen.htm?b=0&boerse=&stellg1=&stellg2=&stellort=&page=2
Waiting for 4.15 seconds before scraping the next page...
Scraping page 3 of 24: https://dlr.concludis.de/prj/lst/a181a603769c1f98ad927e7367c7aa51/GesamtlisteOffenePositionen.htm?b=0&boerse=&stellg1=&stellg2=&stellort=&page=3
Waiting for 3.66 seconds before scraping the next page...
Scraping page 4 of 24: https://dlr.concludis.de/prj/lst/a181a603769c1f98ad927e7367c7aa51/GesamtlisteOffenePositionen.htm?b=0&boerse=&stellg1=&stellg2=&stellort=&page=4
Waiting for 3.13 seconds before scraping the next page...
Scraping page 5 of 24: https://dlr.concludis.de/prj/lst/a181a603769c1f98ad927e7367c7aa51/Gesamtl